In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys, os
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('image', cmap='Blues')
from slmsuite.holography.algorithms import Hologram
from slmsuite.hardware.slms.slm import SLM
from slmsuite.hardware.cameras.camera import Camera
from slmsuite.hardware.cameraslms import FourierSLM
from slmsuite.holography.algorithms import SpotHologram
from slmsuite.holography import analysis
import copy
from IPython.display import Image
import imageio
import cv2
import numpy as np
import ctypes
from ctypes import *
from scipy import misc
from time import sleep
import csv
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
# from instrumental.drivers.cameras import uc480 

from slmfunctions.dependencies import *
from slmfunctions.settings import *
from slmfunctions.simulation import *
from slmfunctions.imageprocessing import *
from slmfunctions.phaseretrieval import *
from slmfunctions.slmsuite.slmsuite.holography.analysis import *

from scipy.optimize import curve_fit


# TURN ON FOR EXPERIMENT

# from slmfunctions.slmsuite.slmsuite.hardware.slms.meadowlark import Meadowlark
# # Meadowlark.info(verbose = True)
# from slmfunctions.slmsuite.slmsuite.hardware.cameras.thorlabs import ThorCam
# ThorCam.info(verbose=True)
# from slmfunctions.slmsuite.slmsuite.hardware.cameraslms import FourierSLM

# GLOBAL VARIABLES
numpixels = 1300
slmwidth = 1200
slmheight = 1920



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:

imgname = f"3by3_9spacing_mag3" # LOOKUP NAME
# load_dir = r'Z:\nsong\NeutralAtoms\Ultracold-Research\GWS-Summer2024\storage'  # For PC
load_dir = r'C:\cleen\nsong\2-Research\UAC Research\Github\Ultracold-Research\GWS-Summer2024\storage' # For laptop
phaseimg_path = os.path.join(load_dir, f"{imgname}_phase.bmp")
virtualintimg_path = os.path.join(load_dir, f"{imgname}_virtualint.jpg")
intimg_path = os.path.join(load_dir, f"{imgname}_int.jpg")

phaseimg = cp.zeros((numpixels, numpixels))
slmphase = cp.array(Image.open(phaseimg_path).convert('L')) / 255*2*cp.pi - cp.pi
phaseimg = slmphase[0:slmwidth, slmheight // 2 - slmwidth // 2: slmheight // 2 + slmwidth // 2]
virtualintensityimg = (cp.array(Image.open(virtualintimg_path).convert('L')) / 255)
intensityimg = (cp.array(Image.open(intimg_path).convert('L')).astype(np.uint8) / 255)

In [ ]:
### For image processing purposes
targetvectors = zoomin(intensityimg)

In [ ]:
from slmsuite.hardware.slms.meadowlark import Meadowlark
slm = Meadowlark(sdk_path="C:\\Program Files\\Meadowlark Optics\\Blink 1920 HDMI", lut_path=r'C:\cleen\nsong\2-Research\UAC Research\Github\Ultracold-Research\GWS-Summer2024\initslm\globalcalibrationat650_june2024.lut')
slm.load_vendor_phase_correction(
    file_path=r'C:\cleen\nsong\2-Research\UAC Research\Github\Ultracold-Research\MeadowLark SLM\SLM\slm6287_at785_WFC.jpg',  # Update this path to load your calibration
)

In [ ]:
from slmsuite.hardware.cameras.thorlabs import ThorCam
ThorCam.info(verbose=True)
cam = ThorCam()

In [ ]:
# Write phase to slm
slm.write(slmphase, settle=True)
# Get camera image
cam.set_exposure(0.02)
image = cam.get_image()



In [ ]:


anchors, angle, Xmag, Ymag, Xshift, Yshift = identify_corners(arr1[100:-100,100:-100], arr1, 50)

target_anchors = writeimg(anchors, f"anchors_{imgname}")

target_im_anchors = np.asarray(Image.open(f"anchors_{imgname}.bmp"))
newtar = np.zeros((1920*precision,1200*precision))

newtar[:,:] = target_im_anchors[:,:,1]
target_im_anchors = newtar / np.max(newtar)


np.savez_compressed(f"anchors_netarray_{imgname}.npz", array1=phase, array2 = PS_shape, array3 = w_prev, array4 = w)
loaded_data = np.load(f"netarray_{imgname}.npz")
phase = loaded_data['array1']
PS_shape = loaded_data['array2']
w_prev = loaded_data['array3']
w = loaded_data['array4']

anchors_loaded_data = np.load(f"anchors_netarray_{imgname}.npz")
anchors_phase = anchors_loaded_data['array1']
anchors_PS_shape = anchors_loaded_data['array2']
anchors_w_prev = anchors_loaded_data['array3']
anchors_w = anchors_loaded_data['array4']

## NOW SHOW THE ANCHORS
anchors_phase_c = 0
anchors_slminput = 0
anchors_phases_1d = (anchors_phase.T).ravel().astype(np.uint8)
anchors_phases_c = np.empty(anchors_phases_1d.shape, dtype=np.uint8, order='C')
anchors_phases_c[:] = anchors_phases_1d
anchors_slminput = anchors_phases_c
slm_lib.Write_image(anchors_slminput.ctypes.data_as(POINTER(c_ubyte)), c_uint(1))
sleep(0.04)       
anchors_camera_test =  cam.grab_image(timeout='100s', copy=True, exposure_time = '2ms')
plt.imshow(anchors_camera_test)
plt.colorbar()
plt.show
identify_tweezers(anchors_camera_test_cut, 80)

anchors_matched, angle, Xmag, Ymag, Xshift, Yshift = match_images(anchors_camera_test_cut, anchors_target_formatching_numpy,80)

# plt.imshow(anchors_matched[500:700,1100:1300] + anchors_target_formatching_numpy[500:700,1100:1300])
plt.imshow(anchors_matched[300:900,1100:1600] + anchors_target_formatching_numpy[300:900,1100:1600])
plt.colorbar()
params = angle, Xmag, Ymag, Xshift, Yshift
### Test to check local image matching works
localtest = np.zeros(np.shape(anchors_camera_test_cut))
localtest[3:-1,3:-1] = anchors_camera_test_cut[0:-4,0:-4]

result = match_images_local(localtest, anchors_target_formatching_numpy, params, 10)

